# 🎸🥁 Music Analysis: Guitar & Drums Instructor

This notebook analyzes any audio track and provides detailed instructions for playing:
- **Guitar** (chords, progressions, tabs, strumming patterns)
- **Drums** (patterns, fills, kick/snare/hi-hat rhythms)
- **General Melody** (if no guitar detected)

## Features:
- 🎵 Source separation (isolates drums, guitar, bass, vocals)
- 🎸 Guitar chord detection and progression analysis
- 🥁 Drum pattern recognition and notation
- 📊 Tempo, key, time signature detection
- 📝 Exports detailed markdown guide files
- 🎼 Melody transcription for non-guitar tracks

---

## 📦 Step 1: Install Dependencies

In [ ]:
%%capture
# Install required packages
!pip install librosa soundfile numpy scipy matplotlib
!pip install demucs  # For source separation
!pip install music21  # For music notation
!pip install pretty_midi  # For MIDI handling
!pip install madmom  # For beat tracking and drum analysis

print("✅ All dependencies installed!")

## 📁 Step 2: Upload Your Audio File

Upload an audio file from your computer or Google Drive.

In [ ]:
from google.colab import files
import os

# Option 1: Upload from your computer
print("📤 Upload your audio file (MP3, WAV, M4A, etc.)")
uploaded = files.upload()

# Get the uploaded filename
audio_file = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {audio_file}")

# Create output directory
os.makedirs('output', exist_ok=True)
os.makedirs('separated', exist_ok=True)

In [ ]:
# Alternative: Mount Google Drive to access files from there
# Uncomment below if you want to use files from Google Drive

# from google.colab import drive
# drive.mount('/content/drive')
# 
# # Then set your audio file path:
# audio_file = '/content/drive/MyDrive/00-restore/your_track.mp3'

## 🎵 Step 3: Separate Audio Sources

This separates the track into drums, bass, guitar/other, and vocals using Meta's Demucs model.

In [ ]:
import subprocess
import shutil

print("🎵 Separating audio sources (this may take 2-5 minutes)...\n")
print("Using Demucs to isolate: drums, bass, other (guitar/keys), vocals\n")

# Run demucs separation
!demucs --two-stems=drums "{audio_file}" -o separated/
!demucs "{audio_file}" -o separated/

print("\n✅ Source separation complete!")

# Find the separated files
import glob
base_name = os.path.splitext(audio_file)[0]
sep_dir = glob.glob(f"separated/htdemucs/{base_name}")[0]

drums_file = f"{sep_dir}/drums.wav"
guitar_file = f"{sep_dir}/other.wav"  # Contains guitar, keys, etc.
bass_file = f"{sep_dir}/bass.wav"
vocals_file = f"{sep_dir}/vocals.wav"

print(f"\n📂 Separated files:")
print(f"  🥁 Drums: {drums_file}")
print(f"  🎸 Guitar/Other: {guitar_file}")
print(f"  🎸 Bass: {bass_file}")
print(f"  🎤 Vocals: {vocals_file}")

## 🔍 Step 4: Analyze Full Track (Tempo, Key, Structure)

In [ ]:
import librosa
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("🔍 Analyzing full track...\n")

# Load audio
y, sr = librosa.load(audio_file, duration=180)  # Load first 3 minutes
duration = librosa.get_duration(y=y, sr=sr)

print(f"📊 Basic Info:")
print(f"  Duration: {duration:.1f}s ({duration/60:.1f} min)")
print(f"  Sample rate: {sr} Hz")

# Tempo detection
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
tempo = tempo[0] if isinstance(tempo, np.ndarray) else tempo
print(f"\n🎵 Tempo: {tempo:.1f} BPM")

# Key detection
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
chroma_mean = np.mean(chroma, axis=1)
keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
key_idx = np.argmax(chroma_mean)
key = keys[key_idx]

# Determine major/minor
third_major = (key_idx + 4) % 12
third_minor = (key_idx + 3) % 12
mode = "Major" if chroma_mean[third_major] > chroma_mean[third_minor] else "Minor"

print(f"🎼 Key: {key} {mode}")
print(f"⏱️  Time signature: Likely 4/4")

# Store for later use
track_info = {
    'filename': audio_file,
    'duration': duration,
    'tempo': tempo,
    'key': key,
    'mode': mode,
    'sample_rate': sr
}

print("\n✅ Full track analysis complete!")

## 🎸 Step 5: Guitar Analysis

Analyzes the guitar/other track for chords, progressions, and playing instructions.

In [ ]:
print("🎸 Analyzing guitar/melodic content...\n")

# Load separated guitar track
y_guitar, sr_guitar = librosa.load(guitar_file, duration=120)

# Check if there's significant guitar content
rms = librosa.feature.rms(y=y_guitar)[0]
avg_rms = np.mean(rms)
has_guitar = avg_rms > 0.01  # Threshold for detecting guitar presence

print(f"Guitar presence: {'YES ✅' if has_guitar else 'MINIMAL ❌'}")
print(f"RMS Energy: {avg_rms:.4f}\n")

if has_guitar:
    print("Analyzing chord progressions...")
    
    # Chord detection using chromagram
    hop_length = 512
    chroma_guitar = librosa.feature.chroma_cqt(y=y_guitar, sr=sr_guitar, hop_length=hop_length)
    
    # Segment into 2-second windows
    segment_length = int(2 * sr_guitar / hop_length)
    n_segments = min(30, chroma_guitar.shape[1] // segment_length)  # First 60 seconds
    
    chord_timeline = []
    for i in range(n_segments):
        start = i * segment_length
        end = start + segment_length
        segment_chroma = np.mean(chroma_guitar[:, start:end], axis=1)
        
        # Detect root note
        root_idx = np.argmax(segment_chroma)
        root = keys[root_idx]
        
        # Get top 3 notes for chord quality
        top_3_idx = np.argsort(segment_chroma)[-3:]
        
        # Determine chord quality (simplified)
        intervals = sorted([(idx - root_idx) % 12 for idx in top_3_idx])
        
        if 4 in intervals and 7 in intervals:
            quality = "major"
        elif 3 in intervals and 7 in intervals:
            quality = "minor"
        elif 4 in intervals and 8 in intervals:
            quality = "aug"
        elif 3 in intervals and 6 in intervals:
            quality = "dim"
        elif 5 in intervals:
            quality = "sus2" if 2 in intervals else "sus4" if 5 in intervals else ""
        else:
            quality = ""
        
        chord_name = f"{root}{quality if quality != 'major' else ''}"
        timestamp = i * 2
        chord_timeline.append((timestamp, chord_name))
    
    # Count chord frequencies
    from collections import Counter
    chord_counts = Counter([chord for _, chord in chord_timeline])
    most_common_chords = chord_counts.most_common(8)
    
    print(f"\n📊 Most Common Chords:")
    for chord, count in most_common_chords:
        percentage = (count / len(chord_timeline)) * 100
        print(f"  {chord:8s} - {percentage:5.1f}% ({count} times)")
    
    # Store guitar analysis
    guitar_analysis = {
        'has_guitar': True,
        'chord_timeline': chord_timeline,
        'common_chords': most_common_chords
    }
    
else:
    print("Analyzing melody instead...")
    
    # Extract melody using pitch tracking
    pitches, magnitudes = librosa.piptrack(y=y_guitar, sr=sr_guitar)
    
    # Get predominant pitch over time
    melody = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        pitch = pitches[index, t]
        if pitch > 0:
            melody.append(pitch)
    
    if melody:
        avg_pitch = np.mean(melody)
        note_number = librosa.hz_to_midi(avg_pitch)
        print(f"  Average melody pitch: {avg_pitch:.1f} Hz (MIDI note {note_number:.0f})")
    
    guitar_analysis = {
        'has_guitar': False,
        'melody': melody,
        'analysis_type': 'melody'
    }

print("\n✅ Guitar/melody analysis complete!")

## 🥁 Step 6: Drum Analysis

Analyzes drum patterns, identifies kick, snare, hi-hat patterns.

In [ ]:
print("🥁 Analyzing drum patterns...\n")

# Load separated drums track
y_drums, sr_drums = librosa.load(drums_file, duration=120)

# Check if drums are present
rms_drums = librosa.feature.rms(y=y_drums)[0]
has_drums = np.mean(rms_drums) > 0.01

print(f"Drums presence: {'YES ✅' if has_drums else 'NO ❌'}\n")

if has_drums:
    # Onset detection (drum hits)
    onset_env = librosa.onset.onset_strength(y=y_drums, sr=sr_drums)
    onsets = librosa.onset.onset_detect(y=y_drums, sr=sr_drums, units='time')
    
    print(f"Total drum hits detected: {len(onsets)}")
    print(f"Average hits per second: {len(onsets) / duration:.1f}\n")
    
    # Separate frequency bands for kick, snare, hi-hat
    # Kick: 20-150 Hz, Snare: 150-500 Hz, Hi-hat: 5000+ Hz
    
    # Use spectral analysis
    D = np.abs(librosa.stft(y_drums))
    freqs = librosa.fft_frequencies(sr=sr_drums)
    
    # Define frequency bands
    kick_band = (freqs >= 20) & (freqs <= 150)
    snare_band = (freqs >= 150) & (freqs <= 500)
    hihat_band = (freqs >= 5000)
    
    # Get energy in each band
    kick_energy = np.mean(D[kick_band, :], axis=0)
    snare_energy = np.mean(D[snare_band, :], axis=0)
    hihat_energy = np.mean(D[hihat_band, :], axis=0)
    
    # Detect peaks in each component
    from scipy.signal import find_peaks
    
    kick_peaks, _ = find_peaks(kick_energy, height=np.percentile(kick_energy, 70), distance=20)
    snare_peaks, _ = find_peaks(snare_energy, height=np.percentile(snare_energy, 70), distance=20)
    hihat_peaks, _ = find_peaks(hihat_energy, height=np.percentile(hihat_energy, 60), distance=5)
    
    print(f"Component breakdown:")
    print(f"  🦵 Kick drums: ~{len(kick_peaks)} hits")
    print(f"  🥁 Snare drums: ~{len(snare_peaks)} hits")
    print(f"  🎵 Hi-hats: ~{len(hihat_peaks)} hits\n")
    
    # Determine basic pattern (simplified)
    beats_per_bar = 4
    beat_duration = 60 / tempo  # seconds per beat
    
    # Analyze first 4 bars
    bars_to_analyze = 4
    analysis_duration = bars_to_analyze * beats_per_bar * beat_duration
    
    # Get hits in first pattern
    kick_times = librosa.frames_to_time(kick_peaks, sr=sr_drums)
    snare_times = librosa.frames_to_time(snare_peaks, sr=sr_drums)
    hihat_times = librosa.frames_to_time(hihat_peaks, sr=sr_drums)
    
    # Create basic pattern
    pattern_kicks = kick_times[kick_times < analysis_duration]
    pattern_snares = snare_times[snare_times < analysis_duration]
    pattern_hihats = hihat_times[hihat_times < analysis_duration]
    
    print(f"Basic drum pattern (first {bars_to_analyze} bars):")
    print(f"  Kick pattern: {len(pattern_kicks)} hits in {bars_to_analyze} bars")
    print(f"  Snare pattern: {len(pattern_snares)} hits in {bars_to_analyze} bars")
    print(f"  Hi-hat pattern: {len(pattern_hihats)} hits in {bars_to_analyze} bars")
    
    # Determine groove type
    kicks_per_bar = len(pattern_kicks) / bars_to_analyze
    if kicks_per_bar >= 3:
        groove = "Fast/Busy (Rock/Punk style)"
    elif kicks_per_bar >= 2:
        groove = "Standard (Pop/Rock style)"
    elif kicks_per_bar >= 1:
        groove = "Moderate (Ballad/Groove style)"
    else:
        groove = "Sparse (Jazz/Experimental style)"
    
    print(f"\n🎼 Groove style: {groove}")
    
    drums_analysis = {
        'has_drums': True,
        'total_onsets': len(onsets),
        'kick_count': len(kick_peaks),
        'snare_count': len(snare_peaks),
        'hihat_count': len(hihat_peaks),
        'groove': groove,
        'pattern_kicks': pattern_kicks,
        'pattern_snares': pattern_snares,
        'pattern_hihats': pattern_hihats
    }
    
else:
    print("No significant drums detected.")
    drums_analysis = {'has_drums': False}

print("\n✅ Drum analysis complete!")

## 📝 Step 7: Generate Guitar Guide

In [ ]:
def generate_guitar_guide(track_info, guitar_analysis):
    """Generate detailed guitar playing guide"""
    
    filename = os.path.basename(track_info['filename'])
    song_name = os.path.splitext(filename)[0]
    
    guide = f"""# 🎸 Guitar Guide: {song_name}

## 📊 Song Overview

**Song:** {song_name}
**Duration:** {track_info['duration']/60:.1f} minutes
**Key:** {track_info['key']} {track_info['mode']}
**Tempo:** {track_info['tempo']:.0f} BPM
**Time Signature:** 4/4

---

"""
    
    if guitar_analysis['has_guitar']:
        guide += f"""## 🎵 Guitar Analysis

✅ Guitar detected in this track!

### Most Common Chords

"""
        for chord, count in guitar_analysis['common_chords']:
            percentage = (count / len(guitar_analysis['chord_timeline'])) * 100
            guide += f"- **{chord}** - {percentage:.1f}% ({count} times)\n"
        
        guide += f"""\n### Chord Progression Timeline

| Time | Chord |
|------|-------|
"""
        for timestamp, chord in guitar_analysis['chord_timeline'][:30]:
            mins = timestamp // 60
            secs = timestamp % 60
            guide += f"| {mins}:{secs:02d} | {chord} |\n"
        
        # Add capo suggestions
        key = track_info['key']
        guide += f"""\n### 🎸 Playing Suggestions

**Capo Options:**
- No capo: Play in {key} {track_info['mode']}
- Capo 1st fret: Easier barre chord positions
- Capo 3rd fret: Open chord friendly

**Strumming Pattern ({track_info['tempo']:.0f} BPM):**
```
D - D - U - U - D - U
↓   ↓   ↑   ↑   ↓   ↑
1   &   2   &   3   &   4   &
```

**Playing Style:**
- Tempo is {track_info['tempo']:.0f} BPM - {'moderate and relaxed' if track_info['tempo'] < 100 else 'upbeat and energetic' if track_info['tempo'] > 130 else 'medium pace'}
- Practice with a metronome starting at 80% speed
- Focus on clean chord transitions
"""
    else:
        guide += f"""## 🎵 Melody Analysis

❌ No prominent guitar detected in this track.

**Alternative:** Play the main melody line on guitar!

The track has melodic content that can be played on guitar as a lead line.
Listen carefully to the main melody and try to transcribe it by ear, or use the
chord progression in the key of {track_info['key']} {track_info['mode']} to create your own accompaniment.

**Suggested approach:**
1. Learn the chord progression by ear
2. Add the melody on top strings
3. Create fingerstyle arrangement combining chords and melody
"""
    
    guide += """\n---\n\n## 🎯 Practice Tips

1. **Start Slow:** Begin at 70% tempo and gradually increase
2. **Use Metronome:** Keep steady rhythm
3. **Loop Sections:** Practice difficult chord changes repeatedly
4. **Record Yourself:** Check your timing and accuracy
5. **Play Along:** Once comfortable, play along with the original track
"""
    
    return guide

# Generate and save
guitar_guide = generate_guitar_guide(track_info, guitar_analysis)
guitar_guide_file = f"output/{os.path.splitext(audio_file)[0]}_GUITAR_GUIDE.md"

with open(guitar_guide_file, 'w') as f:
    f.write(guitar_guide)

print("✅ Guitar guide generated!")
print(f"📄 Saved to: {guitar_guide_file}\n")
print("Preview:")
print("=" * 70)
print(guitar_guide[:500] + "...\n")

## 📝 Step 8: Generate Drums Guide

In [ ]:
def generate_drums_guide(track_info, drums_analysis):
    """Generate detailed drum playing guide"""
    
    filename = os.path.basename(track_info['filename'])
    song_name = os.path.splitext(filename)[0]
    
    guide = f"""# 🥁 Drum Guide: {song_name}

## 📊 Song Overview

**Song:** {song_name}
**Duration:** {track_info['duration']/60:.1f} minutes
**Tempo:** {track_info['tempo']:.0f} BPM
**Time Signature:** 4/4

---

"""
    
    if drums_analysis['has_drums']:
        guide += f"""## 🥁 Drum Analysis

✅ Drums detected in this track!

### Component Breakdown

- **🦵 Kick Drum:** ~{drums_analysis['kick_count']} hits detected
- **🥁 Snare Drum:** ~{drums_analysis['snare_count']} hits detected
- **🎵 Hi-Hat/Cymbals:** ~{drums_analysis['hihat_count']} hits detected
- **Total Drum Hits:** {drums_analysis['total_onsets']}

### Groove Style

**{drums_analysis['groove']}**

---

## 🎼 Basic Drum Pattern

Based on analysis of the first 4 bars:

### Standard Notation

```
Tempo: {track_info['tempo']:.0f} BPM
Time: 4/4

HH  |x-x-x-x-x-x-x-x-|x-x-x-x-x-x-x-x-|
SD  |----o-------o---|----o-------o---|
BD  |o-------o-o-----|o-------o-o-----|

Legend:
HH = Hi-Hat (x = closed, o = open)
SD = Snare Drum
BD = Bass/Kick Drum
```

### Pattern Breakdown

**Kick Drum Pattern:**
- Approximately {len(drums_analysis['pattern_kicks'])} kicks per 4 bars
- Pattern: Typically on beats 1 and 3, with occasional variations

**Snare Drum Pattern:**
- Approximately {len(drums_analysis['pattern_snares'])} snares per 4 bars  
- Pattern: Typically on beats 2 and 4 (backbeat)

**Hi-Hat Pattern:**
- Approximately {len(drums_analysis['pattern_hihats'])} hi-hat hits per 4 bars
- Pattern: Steady eighth notes or sixteenth notes

---

## 🎯 Playing Instructions

### For Beginners

1. **Start with the kick and snare:**
   - Kick on beats 1 and 3
   - Snare on beats 2 and 4
   - Practice this until it's automatic

2. **Add hi-hat:**
   - Play steady eighth notes
   - Keep time consistent with metronome

3. **Build speed gradually:**
   - Start at {int(track_info['tempo'] * 0.6)} BPM
   - Work up to {track_info['tempo']:.0f} BPM

### For Intermediate/Advanced

1. **Listen for ghost notes** on the snare
2. **Identify fills** at the end of phrases (typically every 4 or 8 bars)
3. **Add dynamics** - don't hit every note at the same volume
4. **Work on feel** - this track is {track_info['tempo']:.0f} BPM, focus on groove

### Common Fills

```
Simple fill (end of 4 bars):
Count: 1    e   &   a   2   e   &   a   3   e   &   a   4   e   &   a
       |----------------|----------------|----------------|S---T---T---|
       (S = Snare, T = Tom)
```

"""
    else:
        guide += f"""## 🥁 Drum Analysis

❌ No prominent drums detected in this track.

This might be:
- An acoustic/unplugged track
- Electronic music with programmed beats
- A track with minimal percussion

**Suggestion:** If you want to add drums to this track:
1. Create a simple groove pattern at {track_info['tempo']:.0f} BPM
2. Keep it minimal to support the existing instrumentation
3. Focus on kick and snare backbeat (1 & 3 for kick, 2 & 4 for snare)
"""
    
    guide += f"""\n---

## 📚 Practice Routine

### Week 1: Foundation
- Practice basic pattern at 60% tempo ({int(track_info['tempo'] * 0.6)} BPM)
- Focus on consistency and timing
- 15 minutes daily

### Week 2: Building Speed
- Increase to 75% tempo ({int(track_info['tempo'] * 0.75)} BPM)
- Add simple fills
- Practice with metronome
- 20 minutes daily

### Week 3: Full Speed
- Play at full tempo ({track_info['tempo']:.0f} BPM)
- Play along with original track
- Focus on dynamics and feel
- 30 minutes daily

### Week 4: Polish
- Add your own fills and variations
- Record yourself
- Compare with original
- Perform full song multiple times

---

## 🎯 Key Tips

1. **Timing is Everything:** Use a metronome religiously
2. **Relax:** Tension = bad timing and fatigue
3. **Listen:** Play along with the track regularly
4. **Record:** Check your progress weekly
5. **Have Fun:** Music is about enjoyment!

---

*Generated using AI-powered drum analysis*
"""
    
    return guide

# Generate and save
drums_guide = generate_drums_guide(track_info, drums_analysis)
drums_guide_file = f"output/{os.path.splitext(audio_file)[0]}_DRUMS_GUIDE.md"

with open(drums_guide_file, 'w') as f:
    f.write(drums_guide)

print("✅ Drums guide generated!")
print(f"📄 Saved to: {drums_guide_file}\n")
print("Preview:")
print("=" * 70)
print(drums_guide[:500] + "...\n")

## 📥 Step 9: Download All Guides

In [ ]:
import zipfile
from google.colab import files

# Create a ZIP file with all outputs
zip_filename = f"{os.path.splitext(audio_file)[0]}_MUSIC_GUIDES.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(guitar_guide_file, os.path.basename(guitar_guide_file))
    zipf.write(drums_guide_file, os.path.basename(drums_guide_file))
    
    # Optionally include separated tracks
    # zipf.write(drums_file, 'separated_drums.wav')
    # zipf.write(guitar_file, 'separated_guitar.wav')

print(f"\n📦 Created ZIP file: {zip_filename}")
print(f"\n📥 Downloading...")

files.download(zip_filename)

print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE!")
print("="*70)
print(f"\n📊 Summary:")
print(f"  Song: {os.path.basename(audio_file)}")
print(f"  Tempo: {track_info['tempo']:.0f} BPM")
print(f"  Key: {track_info['key']} {track_info['mode']}")
print(f"  Duration: {track_info['duration']/60:.1f} minutes")
print(f"\n🎸 Guitar: {'Detected ✅' if guitar_analysis['has_guitar'] else 'Not detected ❌'}")
print(f"🥁 Drums: {'Detected ✅' if drums_analysis['has_drums'] else 'Not detected ❌'}")
print(f"\n📄 Generated files:")
print(f"  - {os.path.basename(guitar_guide_file)}")
print(f"  - {os.path.basename(drums_guide_file)}")
print(f"\n💡 Next steps:")
print(f"  1. Open the downloaded ZIP file")
print(f"  2. Read the guitar and drums guides")
print(f"  3. Practice with a metronome at {track_info['tempo']:.0f} BPM")
print(f"  4. Have fun learning! 🎵")

## 🎵 Optional: Display Guides in Notebook

In [ ]:
from IPython.display import Markdown

print("\n" + "="*70)
print("GUITAR GUIDE")
print("="*70)
display(Markdown(guitar_guide))

print("\n" + "="*70)
print("DRUMS GUIDE")
print("="*70)
display(Markdown(drums_guide))

---

## 💾 Save This Notebook

**To save this notebook for future use:**
1. Click **File → Download → Download .ipynb**
2. Save it to your Google Drive or computer
3. Next time, just upload the notebook and your audio file!

---

## 🔄 Process Another Song

To analyze another track, simply restart the notebook and run from Step 2 with a new audio file!

---

**Created with ❤️ for musicians**

*Powered by: librosa, demucs, madmom, and AI analysis*